In [98]:
def parse_ovs_dump_ports_output(output):
    """Parses the output of 'ovs-ofctl dump-ports s1' command and extracts important values."""
    lines = output.splitlines()[1:]
    lines = [' '.join(lines[i:i+2]) for i in range(0,len(lines)-1,2)]

    values = {}
    for line in lines:
        # print(line,'####')
        
        if "port " in line and 'LOCAL' not in line:
            port_name = line.split("port ")[1].split(":")[0].strip()
            port_name = int(port_name[-2])
            values[port_name] = {}
            tmp = line.split('            ')
            # print(tmp)

            rx_packets = int(tmp[0].split("rx pkts=")[1].split(",")[0].strip())
            values[port_name]["rx pkts"] = rx_packets

            rx_bytes = int(tmp[0].split("bytes=")[1].split(",")[0].strip())
            values[port_name]["rx bytes"] = rx_bytes

            tx_packets = int(tmp[1].split("tx pkts=")[1].split(",")[0].strip())
            values[port_name]["tx pkts"] = rx_packets

            tx_bytes = int(tmp[1].split("bytes=")[1].split(",")[0].strip())
            values[port_name]["tx bytes"] = rx_bytes
    return values


In [107]:
pdata = parse_ovs_dump_ports_output(out)

In [93]:
out = 'OFPST_PORT reply (xid=0x2): 4 ports\n  port LOCAL: rx pkts=0, bytes=0, drop=156, errs=0, frame=0, over=0, crc=0\n           tx pkts=0, bytes=0, drop=0, errs=0, coll=0\n  port  "s1-eth1": rx pkts=60, bytes=5638, drop=0, errs=0, frame=0, over=0, crc=0\n           tx pkts=157, bytes=15040, drop=0, errs=0, coll=0\n  port  "s1-eth2": rx pkts=80, bytes=7070, drop=0, errs=0, frame=0, over=0, crc=0\n           tx pkts=151, bytes=14644, drop=0, errs=0, coll=0\n  port  "s1-eth3": rx pkts=79, bytes=6980, drop=0, errs=0, frame=0, over=0, crc=0\n           tx pkts=153, bytes=14824, drop=0, errs=0, coll=0'

In [35]:
print(out)

OFPST_PORT reply (xid=0x2): 4 ports
  port LOCAL: rx pkts=0, bytes=0, drop=156, errs=0, frame=0, over=0, crc=0
           tx pkts=0, bytes=0, drop=0, errs=0, coll=0
  port  1: rx pkts=60, bytes=5638, drop=0, errs=0, frame=0, over=0, crc=0
           tx pkts=157, bytes=15040, drop=0, errs=0, coll=0
  port  2: rx pkts=80, bytes=7070, drop=0, errs=0, frame=0, over=0, crc=0
           tx pkts=151, bytes=14644, drop=0, errs=0, coll=0
  port  3: rx pkts=79, bytes=6980, drop=0, errs=0, frame=0, over=0, crc=0
           tx pkts=153, bytes=14824, drop=0, errs=0, coll=0


In [100]:
tab = 'OFPST_TABLE reply (xid=0x2):\n  table 0:\n    active=1, lookup=217, matched=217\n    max_entries=1000000\n    matching:\n      exact match or wildcard: in_port eth_{src,dst,type} vlan_{vid,pcp} ip_{src,dst} nw_{proto,tos} tcp_{src,dst}\n\n  table 1:\n    active=0, lookup=0, matched=0\n    (same features)\n\n  tables 2...253: ditto'

In [102]:
print(tab.splitlines())

['OFPST_TABLE reply (xid=0x2):', '  table 0:', '    active=1, lookup=217, matched=217', '    max_entries=1000000', '    matching:', '      exact match or wildcard: in_port eth_{src,dst,type} vlan_{vid,pcp} ip_{src,dst} nw_{proto,tos} tcp_{src,dst}', '', '  table 1:', '    active=0, lookup=0, matched=0', '    (same features)', '', '  tables 2...253: ditto']


In [118]:
def get_dump_tables(inp):

    tmp = inp.splitlines()[2]

    active = int(tmp.split("active=")[1].split(",")[0].strip())
    lookup = int(tmp.split("lookup=")[1].split(",")[0].strip())
    matched = int(tmp.split("matched=")[1].split(",")[0].strip())

    
    return [active,lookup,matched]


In [119]:
tdata = get_dump_tables(tab)

In [108]:
deltadic = {1 : {'rx pkts': 0, 'rx bytes': 0, 'tx pkts': 0, 'tx bytes': 0},
          2 : {'rx pkts': 0, 'rx bytes': 0, 'tx pkts': 0, 'tx bytes': 0},
          3 : {'rx pkts': 0, 'rx bytes': 0, 'tx pkts': 0, 'tx bytes': 0}}

In [112]:
deltas = []
for port, data in pdata.items():
    tmp = []
    for f, d in data.items():
        tmp.append(pdata[port][f] - deltadic[port][f])
    deltas.append(tmp)

deltas

[[60, 5638, 60, 5638], [80, 7070, 80, 7070], [79, 6980, 79, 6980]]

In [113]:
pdatalist = []
for port, data in pdata.items():
    tmp = []
    for f, d in data.items():
        tmp.append(pdata[port][f])
    pdatalist.append(tmp)

pdatalist

[[60, 5638, 60, 5638], [80, 7070, 80, 7070], [79, 6980, 79, 6980]]

In [120]:
[pdatalist[i]+deltas[i]+[list(pdata.keys())[i]]+tdata for i in range(len(deltas))]

[[60, 5638, 60, 5638, 60, 5638, 60, 5638, 1, 1, 217, 217],
 [80, 7070, 80, 7070, 80, 7070, 80, 7070, 2, 1, 217, 217],
 [79, 6980, 79, 6980, 79, 6980, 79, 6980, 3, 1, 217, 217]]